In [1]:
def get_gap_group(n, i):
    return PermutationGroup(gap_group = gap.Image(gap.IsomorphismPermGroup(gap.SmallGroup(n, i)))).as_finitely_presented_group()

In [2]:
def list_gap_groups(n):
    for i, H in enumerate(gap.AllSmallGroups(n)):
        print(i + 1, H.StructureDescription())

In [3]:
list_gap_groups(64)

1 C64
2 C8 x C8
3 C8 : C8
4 ((C8 x C2) : C2) : C2
5 (C4 x C2) : C8
6 (C8 x C4) : C2
7 Q8 : C8
8 ((C8 x C2) : C2) : C2
9 (C4 : C4) : C4
10 (C8 : C4) : C2
11 (C2 x C2) . ((C4 x C2) : C2) = (C4 x C2) . (C4 x C2)
12 (C4 : C8) : C2
13 (C2 x C2) . ((C4 x C2) : C2) = (C4 x C2) . (C4 x C2)
14 (C2 x C2) . ((C4 x C2) : C2) = (C4 x C2) . (C4 x C2)
15 C8 : C8
16 C8 : C8
17 (C8 x C2) : C4
18 (C4 x C4) : C4
19 C4 . (C4 x C4)
20 (C4 : C4) : C4
21 (C4 : C4) : C4
22 (C4 x C2) . D8 = C4 . (C4 x C4)
23 ((C4 x C2) : C2) : C4
24 (C8 : C2) : C4
25 (C8 x C2) : C4
26 C16 x C4
27 C16 : C4
28 C16 : C4
29 (C16 x C2) : C2
30 (C16 : C2) : C2
31 (C16 : C2) : C2
32 ((C8 : C2) : C2) : C2
33 (C4 x C2 x C2) : C4
34 ((C2 x C2 x C2) : C4) : C2
35 (C2 x Q8) : C4
36 (C2 . ((C4 x C2) : C2) = (C2 x C2) . (C4 x C2)) : C2
37 C2 . ((C2 x C2 x C2) : C4) = (C4 x C2) . (C4 x C2)
38 (C16 x C2) : C2
39 Q16 : C4
40 (C16 x C2) : C2
41 (C16 : C2) : C2
42 (C16 : C2) : C2
43 C2 . ((C8 x C2) : C2) = C8 . (C4 x C2)
44 C4 : C16
45 C8 . D8 =

G1 = gap("""Group(
 ( 1, 3, 8, 6)(9,33,25,17)(11,35,27,19),
 ( 9,11,16,14)( 1,17,41,40)( 6,22,46,35),
 (17,19,24,22)( 6,25,43,16)( 8,30,41,11)
 );""")

G = PermutationGroup(gap_group = gap.Image(gap.IsomorphismPermGroup(G1))).as_finitely_presented_group()

In [15]:
C2 = groups.presentation.Cyclic(2)
C3 = groups.presentation.Cyclic(3)
C4 = groups.presentation.Cyclic(4)
C6 = groups.presentation.Cyclic(6)
C8 = groups.presentation.Cyclic(8)
C12 = groups.presentation.Cyclic(12)
C16 = groups.presentation.Cyclic(16)
C15 = groups.presentation.Cyclic(15)
#G = C8.direct_product(C4)
# G = groups.presentation.Dihedral(6)
# G = groups.presentation.Symmetric(3).direct_product(C4)
G = get_gap_group(64, 136)

In [16]:
G.structure_description()

'((C2 x Q8) : C2) : C2'

In [17]:
G

Finitely presented group < a, b, c, d, e, f | b^2, c^2, e^2, f^2, d^-2*f, f*a^-2, d*e*d^-1*e, (c*e)^2, a*e*a^-1*e, (c*d^-1)^2, (b*d^-1)^2, a*d^-1*a^-1*d^-1, (b*c)^2, e*c*a*c*a^-1, d*(b*a^-1)^2 >

In [18]:
G.center()

Group([ f ])

In [19]:
# Optimization: Cache index of inverse
# Optimization: Only look at quandling by generators! Perhaps not...

enumed = list(enumerate(G))
#gens = list(G.gens())

rs = G.rewriting_system()
rs.make_confluent()

groupIndex = dict([(rs.reduce(g), i) for i, g in enumed])

def indexOf(g):
    g1 = rs.reduce(g)
    return groupIndex[g1]
    #for i, g1 in enumed:
        #if g == g1:
            #return i

def forOneG(t):
    (i, g) = t
    grelations = []
    
    indexInv = indexOf(g^(-1))
    
    for n in range(len(G)):
        gn = g^n
        if gn == g and n > 1:
            break
        indexN = indexOf(gn)
        grelations.append([indexInv + 1]*n + [indexN + 1])

    for i1, g1 in enumed:
        #i1 = indexOf(g1)
        indexInv1 = indexOf(g1^(-1))
        indexInv2 = indexOf((g1*g*g1^(-1))^(-1)) 
        grelations.append([i1 + 1, i + 1, indexInv1 + 1, indexInv2 + 1])

    return grelations

relations = []

#for i, g in enumed:
    #rel = forOneG(i, g)
    #relations.extend(rel)

from multiprocessing import Pool

pool = Pool(processes = 7)
result = pool.map(forOneG, enumed)
relations = [rel for rels in result for rel in rels]
pool.close()
pool.join()

In [20]:
FreeG = FreeGroup(len(G))

In [21]:
FG = FreeG / relations

In [22]:
FGS = FG.simplified()
FGS

Finitely presented group < x3, x4, x14, x27, x37, x39, x45, x49, x53, x62 | x3^2, x27^2, x4^2, x62^2, x49^2*x53^-2, x53^-2*x39^2, (x49^-1*x53)^2, (x62*x49^-1)^2, x53^4, (x14*x45^-1)^2, x45^-1*x49*x45^-1*x49^-1, (x3*x4)^2, (x62*x3)^2, x27*x53*x27*x53^-1, (x49*x3)^2, x39*x14*x39^-1*x14, x14^-1*x53^-1*x14^-1*x53, (x14^-1*x45^-1)^2, x45*x39^-1*x45*x39, x37^4, (x53*x4)^2, (x49*x27)^2, x53^-1*x3*x53*x3, (x62*x53^-1)^2, (x49*x4)^2, x37*x14*x39^-1*x37^-1*x39^-1*x14, x14*x53*x45^-1*x53^-1*x14^-1*x45^-1, x14*x45^-1*x49^-1*x45*x14^-1*x49, x49^-1*x37*x45*x49*x45^-1*x37^-1, x37^-2*x39^-1*x37^-2*x39, x39*x45*x27*x45^-1*x39^-1*x27, x45*x39*x37^-1*x45^-1*x39^-1*x37, x49*x3*x49^-1*x14*x3*x14, x39^-1*x37^-1*x53^-2*x37*x39^-1, x14^-1*x45*x3*x45^-1*x3*x14^-1, x62*x49^-1*x14*x49^-1*x62*x14^-1, (x39*x45^-1*x49)^2, x14^-1*x39^-1*x14*x53*x39^-1*x53, x27*x53^-1*x62*x27*x62*x53^-1, x14*x39^-1*x53^-1*x39*x14^-1*x53^-1, x39^-1*x14*x49^-1*x14^-1*x39^-1*x49^-1, (x45^-1*x27*x39)^2, x4*x45*x39^-1*x4*x39^-1*x45^-1, x3

In [12]:
FGS.structure_description()

KeyboardInterrupt: 

In [23]:
FGS.order()

16384

In [14]:
FGS1 = FGS

In [24]:
FGS1.is_isomorphic(FGS)

KeyboardInterrupt: 

# Table of computations:

C4 : C4 -> C2 x C2 x C2 x (C4 : C4)

(C4 x C2) : C2 -> C2 x C2 x C4 x ((C4 x C2) : C2)

C2 x D4 -> C2 x C2 x C2 x C2 x C2 x C2 x D4

C2 x Q8 -> C2 x C2 x C2 x C2 x C2 x C2 x Q8

SmallGroup(64, 135) -> something of order 16384

SmallGroup(64, 136) -> something of order 16384

In [28]:
G

Finitely presented group < a, b, c, d, e, f | a^2, b^2, e^2, f^2, d^-2*f, f*c^-2, d*e*d^-1*e, c*e*c^-1*e, (a*e)^2, c*d^-1*c^-1*d^-1, (b*d^-1)^2, (a*d^-1)^2, b*c*b*c^-1, e*c*a*c^-1*a, d^-1*(b*a)^2 >

In [9]:
enumed[6]

(6, a*b^-1)

In [10]:
enumed[36]

(36, a*b^-6)

In [11]:
enumed[39]

(39, b^7)

In [12]:
enumed[41]

(41, a*b^7)

In [47]:
#FG.simplification_isomorphism()

In [58]:
enumed

[(0, 1),
 (1, a),
 (2, b),
 (3, b^-1),
 (4, a*b),
 (5, a*b^-1),
 (6, b^2),
 (7, b^-2),
 (8, a*b^2),
 (9, a*b^-2),
 (10, b^3),
 (11, a*b^3)]

In [50]:
G

Finitely presented group < a, b | a^2, b^16, a^-1*b^-1*a*b >

In [44]:
enumed[1]

(1, a)

In [45]:
enumed[4]

(4, a*b)

In [46]:
enumed[8]

(8, a*b^2)

In [47]:
enumed[16]

(16, a*b^4)

In [48]:
enumed[26]

(26, b^7)

In [49]:
enumed[31]

(31, a*b^8)

In [50]:
enumed[1]

(1, a)

In [51]:
enumed[5]

(5, a*b)

In [52]:
enumed[10]

(10, a^-2*b)

In [27]:
enumed[24]

(24, a^7)

In [15]:
G

Finitely presented group < a, b | a^12, b^2, (a*b)^2 >

In [76]:
DihedralGroup(2)

Dihedral group of order 4 as a permutation group

In [80]:
D3 = DihedralGroup(3) #groups.presentation.Dihedral(3)
C2 = CyclicPermutationGroup(2) #groups.presentation.Cyclic(2)
G1 = C2.direct_product(C2).direct_product(D3)

AttributeError: 'tuple' object has no attribute 'direct_product'

In [81]:
G1.simplification_isomorphism()

AttributeError: 'tuple' object has no attribute 'simplification_isomorphism'

In [91]:
D3p = groups.presentation.Dihedral(3)
D6p = groups.presentation.Dihedral(6)
C2p = groups.presentation.Cyclic(2)
C2p.direct_product(D3p).is_isomorphic(D6p)

True

In [93]:
C2p.direct_product(D6p).structure_description()

'C2 x C2 x S3'

In [87]:
FGS.epimorphisms(DihedralGroup(3))

[Generic morphism:
   From: Finitely presented group < x1, x5, x10 | x5^2, x10^2, (x5*x1)^2, (x5*x10)^2, (x10*x1)^2, x1^6 >
   To:   Dihedral group of order 6 as a permutation group
   Defn: x1 |--> (1,2,3)
         x5 |--> (2,3)
         x10 |--> (2,3)]

In [89]:
groups.presentation.Dihedral(6).epimorphisms(DihedralGroup(3))

[Generic morphism:
   From: Finitely presented group < a, b | a^6, b^2, (a*b)^2 >
   To:   Dihedral group of order 6 as a permutation group
   Defn: a |--> (1,2,3)
         b |--> (2,3)]

In [83]:
FGS.epimorphisms(DihedralGroup(6))

[Generic morphism:
   From: Finitely presented group < x1, x5, x10 | x5^2, x10^2, (x5*x1)^2, (x5*x10)^2, (x10*x1)^2, x1^6 >
   To:   Dihedral group of order 12 as a permutation group
   Defn: x1 |--> (1,2,3,4,5,6)
         x5 |--> (2,6)(3,5)
         x10 |--> (2,6)(3,5), Generic morphism:
   From: Finitely presented group < x1, x5, x10 | x5^2, x10^2, (x5*x1)^2, (x5*x10)^2, (x10*x1)^2, x1^6 >
   To:   Dihedral group of order 12 as a permutation group
   Defn: x1 |--> (1,2,3,4,5,6)
         x5 |--> (1,4)(2,3)(5,6)
         x10 |--> (2,6)(3,5), Generic morphism:
   From: Finitely presented group < x1, x5, x10 | x5^2, x10^2, (x5*x1)^2, (x5*x10)^2, (x10*x1)^2, x1^6 >
   To:   Dihedral group of order 12 as a permutation group
   Defn: x1 |--> (1,3,5)(2,4,6)
         x5 |--> (1,4)(2,3)(5,6)
         x10 |--> (2,6)(3,5)]

In [84]:
groups.presentation.Dihedral(6).epimorphisms(DihedralGroup(6))

[Generic morphism:
   From: Finitely presented group < a, b | a^6, b^2, (a*b)^2 >
   To:   Dihedral group of order 12 as a permutation group
   Defn: a |--> (1,2,3,4,5,6)
         b |--> (2,6)(3,5)]

In [86]:
FGS.epimorphisms(CyclicPermutationGroup(2))

[Generic morphism:
   From: Finitely presented group < x1, x5, x10 | x5^2, x10^2, (x5*x1)^2, (x5*x10)^2, (x10*x1)^2, x1^6 >
   To:   Cyclic group of order 2 as a permutation group
   Defn: x1 |--> ()
         x5 |--> ()
         x10 |--> (1,2), Generic morphism:
   From: Finitely presented group < x1, x5, x10 | x5^2, x10^2, (x5*x1)^2, (x5*x10)^2, (x10*x1)^2, x1^6 >
   To:   Cyclic group of order 2 as a permutation group
   Defn: x1 |--> ()
         x5 |--> (1,2)
         x10 |--> (), Generic morphism:
   From: Finitely presented group < x1, x5, x10 | x5^2, x10^2, (x5*x1)^2, (x5*x10)^2, (x10*x1)^2, x1^6 >
   To:   Cyclic group of order 2 as a permutation group
   Defn: x1 |--> ()
         x5 |--> (1,2)
         x10 |--> (1,2), Generic morphism:
   From: Finitely presented group < x1, x5, x10 | x5^2, x10^2, (x5*x1)^2, (x5*x10)^2, (x10*x1)^2, x1^6 >
   To:   Cyclic group of order 2 as a permutation group
   Defn: x1 |--> (1,2)
         x5 |--> ()
         x10 |--> (), Generic morphism:
  

In [ ]:
G.inject_variables()

In [ ]:
x = b

In [ ]:
y = a*b*a

In [ ]:
indexOf((y*x)^3)

In [ ]:
enumed[8]

In [ ]:
FGS.inject_variables()

In [ ]:
(x10*x3)^6 == x10^0

In [74]:
sage.interfaces.gap.get_gap_memory_pool_size()

799958220

In [75]:
sage.interfaces.gap.set_gap_memory_pool_size(20000000000)